In [17]:
import pandas as pd

df = pd.read_csv("tax_microdata_2021.csv.gz")

In [27]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

def add_pt_w2_wages(df, time_period: int, verbose: bool = True):
    """
    Add pass-through W2 wages to the flat file.

    Args:
        df (pd.DataFrame): The DataFrame to add W2 wages to.

    Returns:
        pd.DataFrame: The DataFrame with W2 wages added.
    """
    qbid_tax_expenditures = { # From JCT TE reports 2018- and 2023-
        2015: 0,
        2016: 0,
        2017: 0,
        2018: 33.2,
        2019: 48.6,
        2020: 56.3,
        2021: 59.0,
        2022: 61.9,
        2023: 55.7,
        2024: 57.6,
        2025: 60.9,
        2026: 24.9,
    }

    QBID_TOTAL_21 = 205.8

    target = QBID_TOTAL_21 * qbid_tax_expenditures[time_period] / qbid_tax_expenditures[2021]

    qbi = np.maximum(0, df.e00900 + df.e26270 + df.e02100 + df.e27200)

    # Solve for scale to match the tax expenditure

    def expenditure_loss(scale):
        res = (qbi * df.s006 * scale[0]).sum()/1e9
        deviation = (res - target)
        if verbose:
            print(f"Scale: {scale[0]}, total: {res}, deviation: {deviation}")
        return deviation ** 2
    
    
    scale = minimize(
        expenditure_loss,
        1,
        tol=1,
    ).x[0]

    df["PT_binc_w2_wages"] = qbi * scale
    
    return df

add_pt_w2_wages(df, 2021).PT_binc_w2_wages

Scale: 1.0, total: 1235.6420345077538, deviation: 1029.8420345077539
Scale: 1.0000000149011612, total: 1235.6420529202553, deviation: 1029.8420529202554
Scale: -0.009999999999999787, total: -12.356420345077279, deviation: -218.1564203450773
Scale: -0.009999985098838593, total: -12.356401932576146, deviation: -218.15640193257616
Scale: 0.16655307989653162, total: 205.79998649688287, deviation: -1.3503117145319266e-05
Scale: 0.1665530947976928, total: 205.80000490938406, deviation: 4.9093840459590865e-06


0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
233384    0.0
233385    0.0
233386    0.0
233387    0.0
233388    0.0
Name: PT_binc_w2_wages, Length: 233389, dtype: float64